In [1]:
#''''''''''''''''''''''''''''''''''''''''''''
# ZanLi
# ID: a1750906
# Introduction-to-Statistical-Machine-Learning
# Assignment: 1
#''''''''''''''''''''''''''''''''''''''''''''

In [20]:
# Importing libraries and modules 

import os

import math as m
import random

import pandas as pd
import numpy as np
from numpy import array

import cvxopt
import cvxopt.solvers
from cvxopt import matrix
from cvxopt import solvers

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

np.random.seed(40)
%matplotlib inline
solvers.options['show_progress'] = False
solvers.options['abstol'] = 1e-10
solvers.options['reltol'] = 1e-10
solvers.options['feastol'] = 1e-10

In [21]:
# Reading the files and converting to necessary labels -1 and 1

X_train = pd.read_csv('./train.csv', header=None)
Y_test = pd.read_csv('./test.csv', header=None)

X_data_train = X_train.iloc[:4000,1:].to_numpy()
y_label_train = X_train.iloc[:4000,0].to_numpy()
y_label_train = np.where(y_label_train == 0, -1, y_label_train)

X_data_val = X_train.iloc[4000:,1:].to_numpy()
y_label_val  = X_train.iloc[4000:,0].to_numpy()
y_label_val = np.where(y_label_val == 0, -1, y_label_val)

Y = Y_test.iloc[:,1:].to_numpy()
y = Y_test.iloc[:,0].to_numpy()
y = np.where(y == 0, -1, y)

In [22]:
# SVM Primal Form - training function

def svm_train_primal(data_train, label_train, regularisation_para_C):
    a, b = data_train.shape
    P_w, P_s, P_b = np.asarray(np.diag(np.ones(b))), np.zeros([b, a]),  np.zeros([1, b]) 
    
    P_1, P_2, P_3, P_4 = np.zeros([a, a]), np.zeros([a, 1]), np.zeros([1, a]), np.zeros([1, 1]) 
    
    P = cvxopt.matrix(np.concatenate((np.concatenate((P_w, P_s, P_b.T), axis = 1), np.concatenate((P_s.T, P_1, P_2), axis=1), np.concatenate((P_b, P_3, P_4),axis = 1)), axis = 0))

    q = cvxopt.matrix((np.concatenate((P_b, np.matrix(np.ones([a]) * regularisation_para_C/a), P_4), axis=1).T))
    
    g_1, g_2, g_3 = np.dot(np.diag(label_train), data_train) * (-1), np.asarray(np.diag(np.ones([a]) * (-1))),  np.matrix(label_train) * (-1)
    g_3 = g_3.T

    G = np.vstack((np.concatenate((g_1,g_2,g_3), axis = 1), np.concatenate((np.zeros([a, b + 1]), np.asarray(np.diag(np.ones([a]) * (-1)))), axis = 1)))
    G = cvxopt.matrix(G) 

    h = cvxopt.matrix(np.hstack((np.ones(a) * (-1), np.zeros(a))))
    
    svm_model = np.array(cvxopt.solvers.coneqp(P,q,G,h)['x']).flatten()
    return svm_model

In [23]:
# SVM Primal Form - testing function

def svm_predict_primal(data_test, label_test, svm_model):
    a, b = X_data_train.shape
    accuracy = accuracy_score(label_test, np.sign(np.dot(data_test, svm_model[:b]) + svm_model[-1]))
    return accuracy

In [24]:
# SVM Primal Form - parameters

svm_model = svm_train_primal(X_data_train , y_label_train , 100)
w_primal = svm_model[:X_data_train.shape[1]]
b_primal = svm_model[-1]
print('w:',np.sum(w_primal),' b:',b_primal)

w: -0.14521568279576713  b: 1.779813670402577


In [ ]:
# SVM Primal Form - accuracy

regularisation_para_C = [2**-10,2**-8,2**-6,2**-4,2**-2,2**0,2**2,2**4,2**6,2**8,2**10]

for k in regularisation_para_C:
    svm_model = svm_train_primal(X_data_train , y_label_train , k)
    test_accuracy_primal = svm_predict_primal(X_data_val , y_label_val, svm_model)
    print('The training accuracy for C = ', k, ' is: ', test_accuracy_primal)

The training accuracy for C =  0.0009765625  is:  0.4908888888888889
The training accuracy for C =  0.00390625  is:  0.4908888888888889


In [18]:
# SVM Dual Form - training function

def svm_train_dual(data_train,  label_train,  regularisation_para_C):
    a, b = data_train.shape
    label_train = label_train.reshape(-1, 1) * 1.

    X_dash = label_train * data_train
    H = np.dot(X_dash , X_dash.T) * 1.
    
    P = matrix(H)
    q = matrix(-np.ones((a, 1)))
    G = matrix(np.vstack((np.eye(a) * (-1), np.eye(a))))
    h = matrix(np.hstack((np.zeros(a), np.ones(a) * (regularisation_para_C/a))))
    A = matrix(label_train.reshape(1, -1))
    b = matrix(np.zeros(1))
    
    res = solvers.qp(P, q, G, h, A, b)
    
    alphas = np.array(res['x'])
    w = ((label_train * alphas).T @ data_train).reshape(-1,1)
    S = (alphas > 1e-4).flatten()
    b = np.mean(label_train[S] - np.dot(data_train[S], w))
    print("alphas:",np.sum(alphas))
    print("w:",np.sum(w))
    print("b:",b)

    svm_model_d = np.vstack((w, b))
    
    return svm_model_d

In [19]:
# SVM Dual Form - parameters

svm_model_d = svm_train_dual(X_data_train , y_label_train , 100)

     pcost       dcost       gap    pres   dres
 0: -3.8416e+02 -2.2800e+02  4e+04  1e+02  5e-13
 1: -2.2204e+01 -2.2367e+02  1e+03  3e+00  5e-13
 2: -1.2704e+01 -1.3798e+02  2e+02  4e-01  7e-14
 3: -8.2452e+00 -4.8671e+01  6e+01  1e-01  3e-14
 4: -6.5928e+00 -2.4543e+01  3e+01  4e-02  2e-14
 5: -5.8533e+00 -1.1273e+01  7e+00  1e-02  2e-14
 6: -5.9382e+00 -7.9755e+00  2e+00  2e-03  2e-14
 7: -6.1146e+00 -6.9517e+00  1e+00  7e-04  2e-14
 8: -6.2067e+00 -6.5700e+00  4e-01  1e-04  2e-14
 9: -6.3032e+00 -6.4094e+00  1e-01  2e-05  2e-14
10: -6.3347e+00 -6.3622e+00  3e-02  4e-06  2e-14
11: -6.3454e+00 -6.3477e+00  2e-03  1e-07  2e-14
12: -6.3464e+00 -6.3465e+00  2e-04  5e-09  2e-14
13: -6.3464e+00 -6.3464e+00  4e-06  1e-10  2e-14
Optimal solution found.
alphas: 7.281637057117683
w: -0.14513522739520301
b: 1.7389327654668323


In [9]:
# Scikit-learn SVM

SVM_classifier = SVC(C = 4/X_data_train.shape[0], kernel = 'linear')
SVM_classifier.fit(X_data_train, y_label_train) 
predicted_labels_train = SVM_classifier.predict(X_data_train)
train_accuracy = accuracy_score(y_label_train, predicted_labels_train)
predicted_labels_test = SVM_classifier.predict(Y)
test_accuracy = accuracy_score(y, predicted_labels_test)
print('w = ',np.sum(SVM_classifier.coef_))
print('b = ',SVM_classifier.intercept_)
print('The training accuracy is : ',train_accuracy)
print('The testing accuracy is : ',test_accuracy)

w =  -0.09878090890038105
b =  [0.91294018]
The training accuracy is :  0.97425
The testing accuracy is :  0.9746666666666667
